In the previous notebook we prepared and massaged our text into input vectors that encode both tokens as well as their positions.  It's these input vectors that will be handled by the rest of the model.

# Self-Attention



Self-Attention is a technique whereby each position in the input sequence can consider the relevancy of each other position in the same sequence when the representation for the sequence is being computed.  (Traditional "attention", as opposed to self-attention look at relations between two different sequences, input- and output, as opposed as using a single sequence.)

## Simplified Self-Attention

Let's take a simple sentence:
"I am learning this"

This is our input sequencen let's call it $x$ with 4 tokens: $x^{(1)}$, $x^{(2)}$, $x^{(3)}$ and $x^{(4)}$.

In general if our context length is $T$ then we have $x^{(1)} \ldots x^{(T)}$

Each $x^{(i)}$ is a $d$-dimensional embedding vector representing a token.

Now we will calculate a context vector $z^{i}$ for each $x^{i}$.  This vector will contain information from all vectors $x^{(1)} \ldots x^{(T)}$